In [51]:
import numpy as np
import pandas as pd
from datetime import datetime
import re
from collections import Counter

In [3]:
def seperate_date_time(x):
    date = pd.to_datetime(x.split(', ')[0], format="%m/%d/%y")
    time = x.split(', ')[1].split(' - ')[0]
    return date, time

In [34]:
def createDf(file_data, dt):
    patterns = {
        '12 Hour':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[APap][mM]\s-\s',
        '24 Hour':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s',
    }
    
    data = ' '.join(file_data.read().split('\n'))
    user_message = re.split(patterns[dt], data)[1:]
    date_time = re.findall(patterns[dt], data)
    
    df = pd.DataFrame({'UserMessage': user_message, 'DateTime': date_time})
    df[['Date','Time']] = df['DateTime'].apply(lambda x: seperate_date_time(x)).to_list()
    users = []
    messages = []
    for message in df['UserMessage']:
        entry = re.split('([\w\W]+?):\s', message)
        if entry[1:]:
            users.append(entry[1])
            messages.append(entry[2])
        else:
            users.append('group_notification')
            messages.append(entry[0])

    df['User'] = users
    df['Message'] = messages
    df.drop(columns=['UserMessage', 'DateTime'], inplace=True)
    return df

In [40]:
f = open('WhatsApp Chat with Say GT (original) 12 hour format.txt', 'r', encoding='utf-8')
df = createDf(f, '12 Hour')

In [36]:
f = open('WhatsApp Chat with Say GT (original) 24 hour format.txt', 'r', encoding='utf-8')
df = pd.DataFrame(createDf(f, '24 Hour'))

In [44]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month_name()
df['MonthNum'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayName'] = df['Date'].dt.day_name()
df['Hour'] = df['Time'].apply(lambda x: x.split(':')[0])
df['Minute'] = df['Time'].apply(lambda x: (x.split(':')[1]).split(' ')[0])

#Uncomment while working with 12 hour file format
# df['Meridian'] = df['Time'].apply(lambda x: (x.split(':')[1]).split(' ')[1])  

df.head()

,Date,Time,User,Message,Year,Month,MonthNum,Day,DayName,Hour,Minute
0,2022-05-09,3:01 PM,group_notification,Messages and calls are end-to-end encrypted. N...,2022,May,5,9,Monday,3,01
1,2022-05-26,7:21 PM,Clg Divyansh,Robotics ke notes,2022,May,5,26,Thursday,7,21
2,2022-05-26,7:44 PM,Clg Kushagra Ahire,MST me jo numerical aaye the uske solution bhe...,2022,May,5,26,Thursday,7,44
3,2022-05-26,7:55 PM,Clg Abbas Amjhera,<Media omitted>,2022,May,5,26,Thursday,7,55
4,2022-05-26,8:05 PM,+91 6239 413 783,Bhai notes bhej do yrr koi 🙏🏻,2022,May,5,26,Thursday,8,05


# Hourly Timeline

In [45]:
def hourly_timeline(selected_user, df, format):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  
  new_df['Message'] = [1] * new_df.shape[0]
  if format == '12 Hour':
    new_df['Hour'] = new_df['Hour'].astype(str) + ' ' + new_df['Meridian'].astype(str)
    new_df = new_df.groupby('Hour').sum().reset_index()
  else:
    new_df = new_df.groupby('Hour').sum().reset_index()
  return new_df

In [46]:
hourly_timeline('Overall', df, '24 Hour')

C:\Users\lenovo\AppData\Local\Temp/ipykernel_14256/631085096.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Message'] = [1] * new_df.shape[0]


,Hour,Message,Year,MonthNum,Day
0,1,210,424620,1338,1270
1,10,109,220398,743,1400
2,11,204,412488,1419,3233
3,12,175,353850,1155,1945
4,2,71,143562,478,1004
5,3,120,242640,746,1838
6,4,71,143562,443,1085
7,5,98,198156,701,904
8,6,39,78858,266,404
9,7,30,60660,196,403


# Daily Timeline

In [47]:
def daily_timeline(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  
  new_df['Message'] = [1] * new_df.shape[0]
  new_df = new_df.groupby('Date').sum().reset_index()
  return new_df

In [48]:
daily_timeline('Overall', df)

C:\Users\lenovo\AppData\Local\Temp/ipykernel_14256/2859091487.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Message'] = [1] * new_df.shape[0]


,Date,Message,Year,MonthNum,Day
0,2022-05-26,16,32352,80,416
1,2022-05-27,14,28308,70,378
2,2022-05-28,5,10110,25,140
3,2022-05-29,8,16176,40,232
4,2022-05-30,37,74814,185,1110
...,...,...,...,...,...
56,2022-08-12,23,46506,184,276
57,2022-08-15,8,16176,64,120
58,2022-08-16,5,10110,40,80
59,2022-08-17,54,109188,432,918


# Weekly Timeline

# Monthly Timeline

# Most Common Word

In [168]:
def most_common_words(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  new_df = new_df[~new_df['Message'].str.contains('<Media omitted>')]
  f = open('stop_hinglish.txt')
  stop_words = f.read()

  words = []

  for message in new_df['Message']:
    for word in message.lower().split():
      if word not in stop_words:
        words.append(word)
  
  most_common_df = pd.DataFrame(Counter(words).most_common(20), columns=['Message', 'Count'])
  most_common_df.sort_values('Count')
  return most_common_df

In [169]:
most_common_words('Overall', df)

,Message,Count
0,message,69
1,deleted,67
2,tcs,42
3,form,25
4,jain,23
5,mail,20
6,digital,19
7,drive,18
8,option,17
9,apply,17
